In [1]:
%env SPECTRAL_CONNECTIVITY_ENABLE_GPU=true

env: SPECTRAL_CONNECTIVITY_ENABLE_GPU=true


In [2]:
import os
os.chdir('/blue/npadillacoreano/mcum/SocialMemEphys/diff_fam_social_memory_ephys')


import cupy as xp
from cupyx.scipy.fft import ifft
from cupyx.scipy.sparse.linalg import svds
from spectral_connectivity import Multitaper, Connectivity
import importlib
import pandas as pd
from itertools import combinations
import os
from bidict import bidict
import lfp.lfp_analysis.LFP_collection as LFP_collection
import pickle 
def pickle_this(thing_to_pickle, file_name):
    """
    Pickles things
    Args (2):
        thing_to_pickle: anything you want to pickle
        file_name: str, filename that ends with .pkl
    Returns:
        none
    """
    with open(file_name, "wb") as file:
        pickle.dump(thing_to_pickle, file)
def unpickle_this(pickle_file):
    """
    Unpickles things
    Args (1):
        file_name: str, pickle filename that already exists and ends with .pkl
    Returns:
        pickled item
    """
    with open(pickle_file, "rb") as file:
        return pickle.load(file)
os.chdir('/blue/npadillacoreano')
df = pd.read_excel(r"share/reward_comp_extention/rce_channel_mapping.xlsx")
spike_cols = [col for col in df.columns if 'spike_interface_' in col.lower()]

# Extract brain regions from column names
# Assumes format 'spike_interface_REGION'
brain_regions = [col.split('spike_interface_')[1] for col in spike_cols]

# Create nested dictionary
subject_to_channel_dict = {}



for _, row in df.iterrows():
    subject = str(row['Subject'])
    # Initialize inner dictionary for this subject
    subject_to_channel_dict[subject] = {}
    
    # Populate inner dictionary with brain region: spike value pairs
    for col, region in zip(spike_cols, brain_regions):
        subject_to_channel_dict[subject][region] = int(row[col])
behavior_dicts = {}
#pickle_this(subject_to_channel_dict[

def make_recording_to_subj_dict(data_path):
    recording_to_subject = {}
    for root, dirs, files in os.walk(data_path):
        for file in files:
            if file.endswith('merged.rec'):
                if file.startswith('2023'):
                    subject = str(file.split("_")[-4].replace('-','.'))
                    recording_to_subject[file] = subject
                    behavior_dicts[file] = {}
                if file.startswith('2024'):
                    subject = str(file.split("_")[-3].replace('-','.'))
                    recording_to_subject[file] = subject
                    behavior_dicts[file] = {}   
                if (file == '20230620_114347_standard_comp_to_omission_D4_subj_1-1_t1b2L_box_2_merged.rec')| (file == '20230620_114347_standard_comp_to_omission_D4_subj_1-2_t3b3L_box_1_merged.rec'):
                    subject = str(file.split("_")[-5].replace('-','.'))
                    recording_to_subject[file] = subject
                    behavior_dicts[file] = {}
                if file =='20230618_100636_standard_comp_to_omission_D2_subj_1_1_t1b2L_box2_merged.rec':
                    recording_to_subject[file] = '1.1'
                if file == '20230618_100636_standard_comp_to_omission_D2_subj_1_4_t4b3L_box1_merged.rec':
                    recording_to_subject[file] = '1.4'
                
    return recording_to_subject

# def process(data_path):
#     recording_to_subject = make_recording_to_subj_dict(data_path)
#     print(recording_to_subject)
#     collection = LFP_collection.LFPCollection(subject_to_channel_dict, data_path, recording_to_subject, 4)
#     #collection.process()
#     return collection    
    




/blue/npadillacoreano/mcum/conda/envs/lfp_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
event_dict_meg = unpickle_this(r"share/reward_comp_extention/event_dict_meg.pkl")
event_dict = unpickle_this(r"share/reward_comp_extention/event_dict.pkl")

print(event_dict.keys())

dict_keys(['20230612_101430_standard_comp_to_training_D1_subj_1-4_t4b2L_box1_merged.rec', '20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.rec', '20230612_112630_standard_comp_to_training_D1_subj_1-2_t2b2L_box1_merged.rec', '20230612_112630_standard_comp_to_training_D1_subj_1-1_t1b3L_box2_merged.rec', '20230613_105657_standard_comp_to_training_D2_subj_1-1_t1b2L_box1_merged.rec', '20230613_105657_standard_comp_to_training_D2_subj_1-4_t4b3L_box2_merged.rec', '20230614_114041_standard_comp_to_training_D3_subj_1-1_t1b3L_box1_merged.rec', '20230614_114041_standard_comp_to_training_D3_subj_1-2_t2b2L_box2_merged.rec', '20230616_111904_standard_comp_to_training_D4_subj_1-4_t4b3L_box1_merged.rec', '20230616_111904_standard_comp_to_training_D4_subj_1-2_t2b2L_box2_merged.rec', '20230617_115521_standard_comp_to_omission_D1_subj_1-1_t1b3L_box1_merged.rec', '20230617_115521_standard_comp_to_omission_D1_subj_1-2_t2b2L_box2_merged.rec', '20230618_100636_standard_comp_to_omissio

In [4]:
data_path = r"share/reward_comp_extention/LFP_rce2_rce3/megadataset/batch2"
recording_to_subject = make_recording_to_subj_dict(data_path)
recordings = list(event_dict.keys())
for key in recording_to_subject.keys():
    if key not in recordings:
        print(key)
#5.3 v 5.5 only have one comp in long comp
#3.1 v 3.4 only have one in alone comp - no long comp since the wrong mice were but in 

# long_comp = 0
# alone_comp= 0
# training = 0
# omission = 0
# for key in recording_to_subject.keys():
#     if 'alone' in key:
#         alone_comp += 1
#     if 'long' in key:
#         long_comp += 1
#     if 'training' in key:
#         training += 1
#     if 'omission' in key:
#         omission += 1
#         print(key)
# print('long comp', long_comp)
# print('alone comp', alone_comp)
# print('training', training)
# print('omission', omission)
                    
                    

In [5]:
#os.chdir('/blue/npadillacoreano/mcum/SocialMemEphys/diff_fam_social_memory_ephys')

collection = LFP_collection.LFPCollection(subject_to_channel_dict, data_path, recording_to_subject, 5, recording_to_event_dict=event_dict)


Processing 20230612_112630_standard_comp_to_training_D1_subj_1-2_t2b2L_box1_merged.rec
Found first timestamp
Processing 20230612_112630_standard_comp_to_training_D1_subj_1-1_t1b3L_box2_merged.rec
Found first timestamp
Processing 20230612_101430_standard_comp_to_training_D1_subj_1-4_t4b2L_box1_merged.rec
Found first timestamp
Processing 20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.rec
Found first timestamp
Processing 20230613_105657_standard_comp_to_training_D2_subj_1-1_t1b2L_box1_merged.rec
Found first timestamp
Processing 20230613_105657_standard_comp_to_training_D2_subj_1-4_t4b3L_box2_merged.rec
Found first timestamp


In [6]:
collection.preprocess(threshold = 5)
collection.calculate_all()


  0%|          | 0/6 [00:00<?, ?it/s]

processing 20230612_112630_standard_comp_to_training_D1_subj_1-2_t2b2L_box1_merged.rec


 17%|█▋        | 1/6 [00:00<00:03,  1.38it/s]

RMS Traces calculated
processing 20230612_112630_standard_comp_to_training_D1_subj_1-1_t1b3L_box2_merged.rec


 33%|███▎      | 2/6 [00:01<00:02,  1.39it/s]

...

Granger causality calculated
Power Calculated
Coherence calcualatd


Maximum iterations reached. 6812 of 6840 converged
Maximum iterations reached. 6814 of 6840 converged
Maximum iterations reached. 6830 of 6840 converged
Maximum iterations reached. 6738 of 6840 converged
Maximum iterations reached. 6802 of 6840 converged
Maximum iterations reached. 6823 of 6840 converged
Maximum iterations reached. 6738 of 6840 converged
Maximum iterations reached. 6828 of 6840 converged
Maximum iterations reached. 6736 of 6840 converged
Maximum iterations reached. 6738 of 6840 converged
 83%|████████▎ | 5/6 [04:00<00:46, 46.82s/it]

Granger causality calculated
Power Calculated
Coherence calcualatd


Maximum iterations reached. 6605 of 6841 converged
Maximum iterations reached. 6603 of 6841 converged
Maximum iterations reached. 6587 of 6841 converged
Maximum iterations reached. 6617 of 6841 converged
Maximum iterations reached. 6747 of 6841 converged
Maximum iterations reached. 6753 of 6841 converged
Maximum iterations reached. 6784 of 6841 converged
Maximum iterations reached. 6747 of 6841 converged
Maximum iterations reached. 6779 of 6841 converged
Maximum iterations reached. 6781 of 6841 converged
100%|██████████| 6/6 [04:46<00:00, 47.71s/it]

Granger causality calculated


In [7]:
# collection.save_to_json(r"share/reward_comp_extention/LFP_rce2_rce3/megadataset/lfp_batch1")

In [ ]:
collection.save_to_json(r"share/reward_comp_extention/LFP_rce2_rce3/megadataset/lfp_batch2")